In [1]:
import cv2
import numpy as np
import os
import subprocess

def run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir, temp_txt='temp_pairs.txt'):
    """
    Precompute SuperGlue matches for all adjacent pairs (i, i+1).
    """
    print("\n=== Running SuperGlue on all adjacent pairs ===")
    with open(temp_txt, 'w') as f:
        for i in range(start_idx, end_idx):
            img1 = f"{i}.jpg"
            img2 = f"{i+1}.jpg"
            f.write(f"{img1} {img2}\n")

    command = [
        "python", "match_pairs.py",
        "--resize", "-1",
        "--superglue", "outdoor",
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--resize_float",
        "--input_dir", input_dir,
        "--input_pairs", temp_txt,
        "--output_dir", output_dir,
        "--viz",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9"
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError("SuperGlue failed:\n" + result.stderr)
    print("✅ SuperGlue matches generated for all adjacent pairs.")


def stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def load_npz_matches(img1, img2):
        filename = f"{img1[:-4]}_{img2[:-4]}_matches.npz"
        npz = np.load(os.path.join(output_dir, filename))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def stitch(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        overlay = result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]]
        mask = (overlay == 0)
        blended = np.where(mask, img1, ((img1.astype(np.float32) + overlay.astype(np.float32)) / 2).astype(np.uint8))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = blended
        return crop_black(result)

    stitched = load_image(f"{start_idx}.jpg")
    for i in range(start_idx, end_idx):
        img1_name = f"{i}.jpg"
        img2_name = f"{i+1}.jpg"
        kp1, kp2 = load_npz_matches(img1_name, img2_name)
        img2 = load_image(img2_name)
        print(f"🔗 Stitching {img1_name} + {img2_name}")
        stitched = stitch(stitched, img2, kp1, kp2)
        cv2.imwrite(os.path.join(input_dir, f"stitched_up_to_{i+1}.jpg"), stitched)

    print(f"\n✅ Final stitched image saved as: stitched_up_to_{end_idx}.jpg")
    return stitched


def run_pipeline(start_idx, end_idx, input_dir, output_dir):
    run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir)
    final_img = stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir)
    return final_img


run_pipeline(1, 48, input_dir="InputImages_ref", output_dir="InputImages_ref/output/")


=== Running SuperGlue on all adjacent pairs ===
✅ SuperGlue matches generated for all adjacent pairs.
🔗 Stitching 1.jpg + 2.jpg
🔗 Stitching 2.jpg + 3.jpg
🔗 Stitching 3.jpg + 4.jpg
🔗 Stitching 4.jpg + 5.jpg
🔗 Stitching 5.jpg + 6.jpg
🔗 Stitching 6.jpg + 7.jpg
🔗 Stitching 7.jpg + 8.jpg
🔗 Stitching 8.jpg + 9.jpg
🔗 Stitching 9.jpg + 10.jpg
🔗 Stitching 10.jpg + 11.jpg
🔗 Stitching 11.jpg + 12.jpg
🔗 Stitching 12.jpg + 13.jpg
🔗 Stitching 13.jpg + 14.jpg
🔗 Stitching 14.jpg + 15.jpg
🔗 Stitching 15.jpg + 16.jpg
🔗 Stitching 16.jpg + 17.jpg
🔗 Stitching 17.jpg + 18.jpg
🔗 Stitching 18.jpg + 19.jpg
🔗 Stitching 19.jpg + 20.jpg
🔗 Stitching 20.jpg + 21.jpg
🔗 Stitching 21.jpg + 22.jpg
🔗 Stitching 22.jpg + 23.jpg
🔗 Stitching 23.jpg + 24.jpg
🔗 Stitching 24.jpg + 25.jpg
🔗 Stitching 25.jpg + 26.jpg
🔗 Stitching 26.jpg + 27.jpg
🔗 Stitching 27.jpg + 28.jpg
🔗 Stitching 28.jpg + 29.jpg
🔗 Stitching 29.jpg + 30.jpg
🔗 Stitching 30.jpg + 31.jpg
🔗 Stitching 31.jpg + 32.jpg
🔗 Stitching 32.jpg + 33.jpg
🔗 Stitching 33.jpg

array([[[105, 141, 147],
        [ 93, 131, 136],
        [ 93, 132, 137],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[139, 184, 191],
        [117, 164, 171],
        [109, 158, 164],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[143, 187, 194],
        [124, 168, 176],
        [109, 155, 162],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [8]:
import cv2
import numpy as np
import os
import subprocess

def run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir, temp_txt='temp_pairs.txt'):
    """
    Precompute SuperGlue matches for all adjacent pairs (i, i+1).
    """
    print("\n=== Running SuperGlue on all adjacent pairs ===")
    with open(temp_txt, 'w') as f:
        for i in range(start_idx, end_idx):
            img1 = f"{i}.jpg"
            img2 = f"{i+1}.jpg"
            f.write(f"{img1} {img2}\n")

    command = [
        "python", "match_pairs.py",
        "--resize", "-1",
        "--superglue", "outdoor",
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--resize_float",
        "--input_dir", input_dir,
        "--input_pairs", temp_txt,
        "--output_dir", output_dir,
        "--viz",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9"
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError("SuperGlue failed:\n" + result.stderr)
    print("✅ SuperGlue matches generated for all adjacent pairs.")


def stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def load_npz_matches(img1, img2):
        filename = f"{img1[:-4]}_{img2[:-4]}_matches.npz"
        npz = np.load(os.path.join(output_dir, filename))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def stitch(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    stitched = load_image(f"{start_idx}.jpg")
    for i in range(start_idx, end_idx):
        img1_name = f"{i}.jpg"
        img2_name = f"{i+1}.jpg"
        kp1, kp2 = load_npz_matches(img1_name, img2_name)
        img2 = load_image(img2_name)
        print(f"🔗 Stitching {img1_name} + {img2_name}")
        stitched = stitch(stitched, img2, kp1, kp2)
        cv2.imwrite(os.path.join(input_dir, f"stitched_up_to_{i+1}.jpg"), stitched)

    print(f"\n✅ Final stitched image saved as: stitched_up_to_{end_idx}.jpg")
    return stitched


def run_pipeline(start_idx, end_idx, input_dir, output_dir):
    run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir)
    final_img = stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir)
    return final_img


run_pipeline(1, 48, input_dir="cropped", output_dir="cropped/output/")


=== Running SuperGlue on all adjacent pairs ===
✅ SuperGlue matches generated for all adjacent pairs.
🔗 Stitching 1.jpg + 2.jpg
🔗 Stitching 2.jpg + 3.jpg
🔗 Stitching 3.jpg + 4.jpg
🔗 Stitching 4.jpg + 5.jpg
🔗 Stitching 5.jpg + 6.jpg
🔗 Stitching 6.jpg + 7.jpg
🔗 Stitching 7.jpg + 8.jpg
🔗 Stitching 8.jpg + 9.jpg
🔗 Stitching 9.jpg + 10.jpg
🔗 Stitching 10.jpg + 11.jpg
🔗 Stitching 11.jpg + 12.jpg
🔗 Stitching 12.jpg + 13.jpg
🔗 Stitching 13.jpg + 14.jpg
🔗 Stitching 14.jpg + 15.jpg
🔗 Stitching 15.jpg + 16.jpg
🔗 Stitching 16.jpg + 17.jpg
🔗 Stitching 17.jpg + 18.jpg
🔗 Stitching 18.jpg + 19.jpg
🔗 Stitching 19.jpg + 20.jpg
🔗 Stitching 20.jpg + 21.jpg
🔗 Stitching 21.jpg + 22.jpg
🔗 Stitching 22.jpg + 23.jpg
🔗 Stitching 23.jpg + 24.jpg
🔗 Stitching 24.jpg + 25.jpg
🔗 Stitching 25.jpg + 26.jpg
🔗 Stitching 26.jpg + 27.jpg
🔗 Stitching 27.jpg + 28.jpg
🔗 Stitching 28.jpg + 29.jpg
🔗 Stitching 29.jpg + 30.jpg
🔗 Stitching 30.jpg + 31.jpg
🔗 Stitching 31.jpg + 32.jpg
🔗 Stitching 32.jpg + 33.jpg
🔗 Stitching 33.jpg

array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 9, 13, 13],
        [35, 51, 52],
        [34, 50, 50],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[12, 17, 17],
        [46, 67, 68],
        [45, 66, 67],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)

In [9]:
import cv2
import numpy as np
import os
import subprocess

def run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir, temp_txt='temp_pairs.txt'):
    """
    Precompute SuperGlue matches for all adjacent pairs (i, i+1).
    """
    print("\n=== Running SuperGlue on all adjacent pairs ===")
    with open(temp_txt, 'w') as f:
        for i in range(start_idx, end_idx):
            img1 = f"{i}.jpg"
            img2 = f"{i+1}.jpg"
            f.write(f"{img1} {img2}\n")

    command = [
        "python", "match_pairs.py",
        "--resize", "-1",
        "--superglue", "outdoor",
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--resize_float",
        "--input_dir", input_dir,
        "--input_pairs", temp_txt,
        "--output_dir", output_dir,
        "--viz",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9"
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError("SuperGlue failed:\n" + result.stderr)
    print("✅ SuperGlue matches generated for all adjacent pairs.")


def stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def load_npz_matches(img1, img2):
        filename = f"{img1[:-4]}_{img2[:-4]}_matches.npz"
        npz = np.load(os.path.join(output_dir, filename))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def stitch(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    stitched = load_image(f"{start_idx}.jpg")
    for i in range(start_idx, end_idx):
        img1_name = f"{i}.jpg"
        img2_name = f"{i+1}.jpg"
        kp1, kp2 = load_npz_matches(img1_name, img2_name)
        img2 = load_image(img2_name)
        print(f"🔗 Stitching {img1_name} + {img2_name}")
        stitched = stitch(stitched, img2, kp1, kp2)
        cv2.imwrite(os.path.join(input_dir, f"stitched_up_to_{i+1}.jpg"), stitched)

    print(f"\n✅ Final stitched image saved as: stitched_up_to_{end_idx}.jpg")
    return stitched


def run_pipeline(start_idx, end_idx, input_dir, output_dir):
    run_superglue_on_all_pairs(start_idx, end_idx, input_dir, output_dir)
    final_img = stitch_images_sequentially(start_idx, end_idx, input_dir, output_dir)
    return final_img


run_pipeline(1, 48, input_dir="Set-01", output_dir="Set-01/output/")


=== Running SuperGlue on all adjacent pairs ===
✅ SuperGlue matches generated for all adjacent pairs.
🔗 Stitching 1.jpg + 2.jpg
🔗 Stitching 2.jpg + 3.jpg
🔗 Stitching 3.jpg + 4.jpg
🔗 Stitching 4.jpg + 5.jpg
🔗 Stitching 5.jpg + 6.jpg
🔗 Stitching 6.jpg + 7.jpg
🔗 Stitching 7.jpg + 8.jpg
🔗 Stitching 8.jpg + 9.jpg
🔗 Stitching 9.jpg + 10.jpg
🔗 Stitching 10.jpg + 11.jpg
🔗 Stitching 11.jpg + 12.jpg
🔗 Stitching 12.jpg + 13.jpg
🔗 Stitching 13.jpg + 14.jpg
🔗 Stitching 14.jpg + 15.jpg
🔗 Stitching 15.jpg + 16.jpg
🔗 Stitching 16.jpg + 17.jpg
🔗 Stitching 17.jpg + 18.jpg
🔗 Stitching 18.jpg + 19.jpg
🔗 Stitching 19.jpg + 20.jpg
🔗 Stitching 20.jpg + 21.jpg
🔗 Stitching 21.jpg + 22.jpg
🔗 Stitching 22.jpg + 23.jpg
🔗 Stitching 23.jpg + 24.jpg
🔗 Stitching 24.jpg + 25.jpg
🔗 Stitching 25.jpg + 26.jpg
🔗 Stitching 26.jpg + 27.jpg
🔗 Stitching 27.jpg + 28.jpg
🔗 Stitching 28.jpg + 29.jpg
🔗 Stitching 29.jpg + 30.jpg
🔗 Stitching 30.jpg + 31.jpg
🔗 Stitching 31.jpg + 32.jpg
🔗 Stitching 32.jpg + 33.jpg
🔗 Stitching 33.jpg

array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [13, 18, 19],
        [23, 34, 34],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [25, 36, 37],
        [46, 67, 68],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)